In [10]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preprocesado de datos

In [11]:

# Cargar datos (asumiendo CSV, si el formato es diferente avísame)
df = pd.read_csv("../0.data/casas.trn.txt", sep='\s+' , header=None)

# Separar características (X) y etiquetas (y)
X = df.iloc[:, :-1].values  # Todas las columnas menos la última
y = df.iloc[:, -1].values   # Última columna (precio de la vivienda)

# Normalizar los datos (media 0, varianza 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convertir a tensores de PyTorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)

# Dividir en conjuntos de entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [35]:
import torch.nn as nn  # Importamos el módulo de redes neuronales de PyTorch

# Definimos la clase del perceptrón, heredando de nn.Module
class Perceptron(nn.Module):
    def __init__(self, input_size, hidden_size):
        """
        Constructor del perceptrón.
        
        Parámetros:
        - input_size: Número de entradas (cantidad de características de cada muestra).
        - hidden_size: Número de neuronas en la capa oculta.
        """
        super(Perceptron, self).__init__()  # Inicializa nn.Module para registrar parámetros
        
        # Definimos la capa oculta (fully connected)
        self.hidden = nn.Linear(input_size, hidden_size)  
        self.f_hidden = nn.Tanh()

        # Definimos la capa de salida con 1 neurona (para regresión)
        self.output = nn.Linear(hidden_size, 1)
        # Esto es la función identidad, realmente no haria falta ponerla
        # en forward simplemnte no le aplicamos ninguna función solo la capa
        self.f_output = lambda x: x

    def forward(self, x):
        """
        Define el flujo de datos a través de la red neuronal.
        
        Parámetro:
        - x: Tensor de entrada (muestra o batch de datos)
        
        Retorna:
        - La predicción del modelo (salida de la capa final)
        """
        x = self.f_hidden(self.hidden(x))  # Aplicamos la capa oculta + activación Tanh
        x = self.f_output(self.output(x))  # Aplicamos la capa de salida sin activación
        return x  # Devolvemos la salida final del modelo

# Crear el modelo con los tamaños adecuados
input_size = X.shape[1]  # Número de características de entrada (columnas del dataset)
hidden_size = 3  # Número de neuronas en la capa oculta (valor arbitrario)

model = Perceptron(input_size, hidden_size)  # Instanciamos el modelo


In [ ]:
# Argumento para saer en que estado esta el modelo train o eval
model.training
# model.eval()
# model.training

True

In [13]:
import torch.optim as optim

criterion = nn.MSELoss()  # Error cuadrático medio
optimizer = optim.SGD(model.parameters(), lr=0.002)  # Descenso de gradiente


In [16]:
epochs = 10000  # Número de iteraciones

for epoch in range(epochs):
    model.train()  # Modo entrenamiento
    optimizer.zero_grad()  # Resetear gradientes
    
    output = model(X_train)  # Forward pass
    loss = criterion(output, y_train)  # Calcular pérdida
    loss.backward()  # Backpropagation
    optimizer.step()  # Actualizar pesos

    if epoch % 100 == 0:
        print(f"Época {epoch}, Pérdida: {loss.item():.4f}")


Época 0, Pérdida: 17.8371
Época 100, Pérdida: 16.2624
Época 200, Pérdida: 14.9660
Época 300, Pérdida: 13.9790
Época 400, Pérdida: 13.2646
Época 500, Pérdida: 12.7349
Época 600, Pérdida: 12.3298
Época 700, Pérdida: 12.0092
Época 800, Pérdida: 11.7488
Época 900, Pérdida: 11.5331
Época 1000, Pérdida: 11.3519
Época 1100, Pérdida: 11.1976
Época 1200, Pérdida: 11.0644
Época 1300, Pérdida: 10.9480
Época 1400, Pérdida: 10.8451
Época 1500, Pérdida: 10.7532
Época 1600, Pérdida: 10.6705
Época 1700, Pérdida: 10.5955
Época 1800, Pérdida: 10.5271
Época 1900, Pérdida: 10.4644
Época 2000, Pérdida: 10.4069
Época 2100, Pérdida: 10.3538
Época 2200, Pérdida: 10.3048
Época 2300, Pérdida: 10.2595
Época 2400, Pérdida: 10.2174
Época 2500, Pérdida: 10.1784
Época 2600, Pérdida: 10.1420
Época 2700, Pérdida: 10.1081
Época 2800, Pérdida: 10.0765
Época 2900, Pérdida: 10.0469
Época 3000, Pérdida: 10.0191
Época 3100, Pérdida: 9.9931
Época 3200, Pérdida: 9.9687
Época 3300, Pérdida: 9.9458
Época 3400, Pérdida: 9.9242
É

In [23]:
model.eval()  # Cambiar a modo evaluación
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)

print(f"Error cuadrático medio en test: {test_loss.item():.4f}")


Error cuadrático medio en test: 510.6031


False